## Importing modules

In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from groq import Groq
import os
import streamlit as st
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import PromptTemplate,load_prompt


In [2]:
load_dotenv()


True

## loading the model

In [4]:
model = ChatGroq(
        groq_api_key=os.getenv("GROQ_API_KEY"),
        model_name="Gemma2-9b-It",
        temperature=0 ## this is creative parameter
    )


## Prompt template example

In [5]:
template2 = PromptTemplate(
    template='Greet this person in 5 languages. The name of the person is {name}',
    input_variables=['name']
)

# fill the values of the placeholders
prompt = template2.invoke({'name':'chinu'})

result = model.invoke(prompt)

print(result.content)

Here are greetings for Chinu in 5 languages:

1. **English:** Hello, Chinu!
2. **Spanish:** Hola, Chinu!
3. **French:** Bonjour, Chinu!
4. **German:** Hallo, Chinu!
5. **Japanese:** こんにちは、チヌさん (Konnichiwa, Chinu-san) 


Let me know if you'd like greetings in other languages! 



## Prompt Generator

In [3]:
# template
template = PromptTemplate(
    template="""
Please summarize the research paper titled "{paper_input}" with the following specifications:
Explanation Style: {style_input}  
Explanation Length: {length_input}  
1. Mathematical Details:  
   - Include relevant mathematical equations if present in the paper.  
   - Explain the mathematical concepts using simple, intuitive code snippets where applicable.  
2. Analogies:  
   - Use relatable analogies to simplify complex ideas.  
If certain information is not available in the paper, respond with: "Insufficient information available" instead of guessing.  
Ensure the summary is clear, accurate, and aligned with the provided style and length.
""",
input_variables=['paper_input', 'style_input','length_input'],
validate_template=True
)

template.save('template.json')

## Prompt UI

In [ ]:
st.header('Reasearch Tool')

paper_input = st.selectbox( "Select Research Paper Name", ["Attention Is All You Need", "BERT: Pre-training of Deep Bidirectional Transformers", "GPT-3: Language Models are Few-Shot Learners", "Diffusion Models Beat GANs on Image Synthesis"] )

style_input = st.selectbox( "Select Explanation Style", ["Beginner-Friendly", "Technical", "Code-Oriented", "Mathematical"] ) 

length_input = st.selectbox( "Select Explanation Length", ["Short (1-2 paragraphs)", "Medium (3-5 paragraphs)", "Long (detailed explanation)"] )

template = load_prompt('template.json') ## loading prompt template by running prompt_generator.py



if st.button('Summarize'):
    chain = template | model
    result = chain.invoke({
        'paper_input':paper_input,
        'style_input':style_input,
        'length_input':length_input
    })
    st.write(result.content)


## Chat bot

In [ ]:
old_style = False

chat_history = []

if old_style:
    while True:
        user_input = input('You: ')
        chat_history.append(user_input) ## we are appending the every hunman response to the chat history
        if user_input == 'exit':
            break
        result = model.invoke(chat_history) ## getting the result
        chat_history.append(result.content)## now we are appending the AI response to the chat history
        print("AI: ",result.content)

    print("----------------------------------------------------")
    print(chat_history)
else:
    # creating a chat history
    chat_history = [
        SystemMessage(content='You are a helpful AI assistant')
    ]

    while True:
        user_input = input('You: ')
        chat_history.append(HumanMessage(content=user_input)) ## we are appending the every hunman response to the chat history
        if user_input == 'exit':
            break
        result = model.invoke(chat_history) ## getting the result
        chat_history.append(AIMessage(content=result.content))## now we are appending the AI response to the chat history
        print("AI: ",result.content)

    print(chat_history)

## Chat prompt template

In [9]:
chat_template = ChatPromptTemplate([
    SystemMessage(content='You are a helpful {domain} expert'),
    HumanMessage(content='Explain in simple terms, what is {topic}')
])
##  output of the above template its not taking place hodlers
# messages=[SystemMessage(content='You are a helpful {domain} expert', additional_kwargs={},
#  response_metadata={}), HumanMessage(content='Explain in simple terms, what is {topic}',
#   additional_kwargs={}, response_metadata={})]
## passing system message like this to over come error
chat_template = ChatPromptTemplate([
    ('system', 'You are a helpful {domain} expert'),
    ('human', 'Explain in simple terms, what is {topic}')
])

prompt = chat_template.invoke({'domain':'cricket','topic':'Dusra'})

print(prompt)

messages=[SystemMessage(content='You are a helpful cricket expert', additional_kwargs={}, response_metadata={}), HumanMessage(content='Explain in simple terms, what is Dusra', additional_kwargs={}, response_metadata={})]


## Messages

In [10]:
model = ChatGroq(
        groq_api_key=os.getenv("GROQ_API_KEY"),
        model_name="Gemma2-9b-It",
        temperature=0.5 ## this is creative parameter
    )

messages=[
    SystemMessage(content='You are a helpful assistant'),
    HumanMessage(content='Tell me about LangChain')
]

result = model.invoke(messages)

messages.append(AIMessage(content=result.content))

print(messages)

[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me about LangChain', additional_kwargs={}, response_metadata={}), AIMessage(content='Let\'s talk about LangChain! \n\nLangChain is an open-source framework designed to simplify the development of applications powered by large language models (LLMs). Think of it as a toolbox filled with components specifically tailored for working with LLMs like me. \n\nHere\'s a breakdown of what makes LangChain so powerful:\n\n**Key Features:**\n\n* **Modular Design:** LangChain breaks down LLM applications into smaller, manageable pieces called "chains." These chains can be composed and chained together to create complex workflows.\n\n* **Variety of Components:** It offers a rich library of pre-built components:\n    * **Models:**  Connectors to various LLMs (e.g., OpenAI, HuggingFace).\n    * **Prompts:** Tools for crafting effective prompts to guide LLM responses.\n    * **M

## Message Place holder

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# chat template
chat_template = ChatPromptTemplate([
    ('system','You are a helpful customer support agent'),
    MessagesPlaceholder(variable_name='chat_history'), ## we pass chat history of the customer here
    ('human','{query}')
])

chat_history = []
# load chat history
with open('chat_history.txt') as f:
    chat_history.extend(f.readlines())

print(chat_history)

# create prompt
prompt = chat_template.invoke({'chat_history':chat_history, 'query':'Where is my refund'})

print(prompt)

['HumanMessage(content="I want to request a refund for my order #12345.")\n', 'AIMessage(content="Your refund request for order #12345 has been initiated. It will be processed in 3-5 business days.")']
messages=[SystemMessage(content='You are a helpful customer support agent', additional_kwargs={}, response_metadata={}), HumanMessage(content='HumanMessage(content="I want to request a refund for my order #12345.")\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='AIMessage(content="Your refund request for order #12345 has been initiated. It will be processed in 3-5 business days.")', additional_kwargs={}, response_metadata={}), HumanMessage(content='Where is my refund', additional_kwargs={}, response_metadata={})]
